In [ ]:
import s4sim.hardware.config
c=s4sim.hardware.config.get_example()

In [ ]:
import s4sim

In [ ]:
s4sim.__version__

In [ ]:
c.data["telescopes"]["SAT0"]["fwhm"]

In [ ]:
c.data["telescopes"]["SAT3"]["fwhm"].keys()

In [ ]:
c.data["telescopes"]["SAT3"].keys()

In [ ]:
c.data["bands"].keys()

In [ ]:
c.data["bands"]["LFL1"]

In [ ]:
import numpy as np

# NET and number of detectors per tube

From the spreadsheet at https://docs.google.com/spreadsheets/d/1X0x8wYhjHdI_WPjhgbtyVlVQrX86tTgxgqEUhan2Hxg/edit?pli=1#gid=1404929524

Extracted on July 2nd 2020

In [ ]:
NUM_DET = {}
SAT_NUM_DET_STRING = "296	296	3556	3556	3556	3556	8594	8594"
for sat_band, num_det in zip(c.data["telescopes"]["SAT3"]["fwhm"].keys(), SAT_NUM_DET_STRING.split()):
    NUM_DET[sat_band] = int(num_det)

In [ ]:
NET = {}
SAT_NET_STRING = "177	224	270	309	238	331	747	1281"
for band, net in zip(c.data["telescopes"]["SAT3"]["fwhm"].keys(), SAT_NET_STRING.split()):
    NET[band] = float(net)

In [ ]:
LAT_NUM_DET_STRING = "216	384	384	3456	3456	3456	3456"
for sat_band, num_det in zip(c.data["telescopes"]["LAT0"]["fwhm"].keys(), LAT_NUM_DET_STRING.split()):
    NUM_DET[sat_band] = int(num_det)

In [ ]:
c.data["telescopes"]["LAT0"]["fwhm"].keys()

In [ ]:
LAT_NET_STRING = "473.0	429.0	261.0	305.0	385.0	792.0	1990.0"
for band, net in zip(c.data["telescopes"]["LAT0"]["fwhm"].keys(), LAT_NET_STRING.split()):
    NET[band] = float(net)

In [ ]:

pole_NET = {}
pole_LAT_NET_STRING = "473.0 445.0 320.0   347.0   374.0   664.0   1609.0"
for band, net in zip(list(c.data["telescopes"]["LAT0"]["fwhm"].keys()), pole_LAT_NET_STRING.split()):
    pole_NET[band] = float(net)

In [ ]:
NET

In [ ]:
pole_NET

In [ ]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [ ]:
ipac_warning = ["Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')"]

In [ ]:
s4 = []
for i, (band, v) in enumerate(c.data["bands"].items()):
    print("## " + band)
    print("Bandpass [GHz] low,center,high: ", v["low"], v["center"], v["high"])
    try:
        tel = "SAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    except:
        tel = "LAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    print("Beam [arcmin]: ", fwhm)
    d = OrderedDict(band=band)
    d["band"] = band
    d["telescope"] = tel
    d["center_frequency"] = v["center"] * u.GHz
    d["fwhm"] = fwhm * u.arcmin
    d["detectors_per_tube"] = NUM_DET[band]
    d["NET"] = NET[band] * u.uK*u.s**.5
    if tel == "LAT":
        d["pole_NET"] = pole_NET[band]* u.uK* u.s**.5
    bandpass_table = QTable(dict(bandpass_frequency=np.linspace(v["low"], v["high"], 10) * u.GHz,
    bandpass_weight=np.ones(10, dtype=np.float)))
    bandpass_table.meta["comments"] = ipac_warning
    bandpass_table.write(f"../instrument_model/bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)
    s4.append(d)

In [ ]:
cat ../instrument_model/bandpass_HFL1.tbl

# Sensitivity from Decadal Survey Report

For verification purposes we want to compare simulation results to the expected performance as detailed in the DSR.
The DSR is available in tex format in the private Github repository: https://github.com/CMB-S4/DecadalSurveyReport

Numbers extracted on 26 January 2021 from version `83d362e2301b80444ed09c29293998cf5917d899`

Changed the frequency labels for LAT

In [ ]:
from io import StringIO
import pandas as pd

def parse_table(table_string):
    """Parse Latex table into a Pandas Dataframe"""
    table = pd.read_csv(StringIO(table_string), sep="&", index_col=0, skipinitialspace=True).T
    table.index = pd.to_numeric(table.index.str.strip())
    table.columns = table.columns.str.strip()
    return table

[SAT](https://github.com/CMB-S4/DecadalSurveyReport/blob/83d362e2301b80444ed09c29293998cf5917d899/ScienceMeasurement/sciencemeasurement.tex#L336-L344), Table `2-1`

In [ ]:
table21 = parse_table("""Frequency & 20 & 30 & 40 & 85 & 95 & 145 & 155 & 220 & 270
Angular resolution (arcmin) & 11.0 & 72.8 & 72.8 & 25.5 &  22.7 & 25.5 & 22.7 & 13.0 & 13.0
Total survey weight / $10^6$ ($\mu$K$^{-2}$) & 0.12& 0.69 & 0.43 & 11.0 & 14.1& 5.7 & 4.8 & 0.71 & 0.24
$Q/U$ rms ($\mu$K-arcmin)  & 8.4 & 3.5 & 4.5 & 0.88 & 0.78 & 1.2 & 1.3 & 3.5 & 6.0""")

[Delensing survey LAT Pole](https://github.com/CMB-S4/DecadalSurveyReport/blob/master/ScienceMeasurement/sciencemeasurement.tex#L445-L448)

In [ ]:
table22 = parse_table("""Frequency (GHz) & 20 & 27 & 39 & 93 & 145 & 225 & 278
Angular resolution (arcmin) & 11.0 & 7.3 & 5.5 & 2.3 & 1.5 & 1.0 & 0.8
White noise level $E$/$B$ ($\mu$K-arcmin) & 8.4 & 5.0 & 4.5 & 0.68 & 0.96 & 5.7 & 9.8""")

[LAT Chile](https://github.com/CMB-S4/DecadalSurveyReport/blob/master/ScienceMeasurement/sciencemeasurement.tex#L523-L530)

In [ ]:
table23 = parse_table("""Frequency (GHz) & 27 & 39 & 93 & 145 & 225 & 278
Angular resolution (arcmin) & 7.4 & 5.1 & 2.2 & 1.4 & 1.0 & 0.9 
Total survey weight ($TT$)/$10^6$ [$\mu$K$^{-2}$]  & 0.22 & 0.68 & 26.3 & 26.3 & 2.2 & 0.38 
White noise level for $TT$ ($\mu$K-arcmin) &21.8 &  12.4&   2.0&    2.0&    6.9&    16.7
White noise level $E$/$B$ ($\mu$K-arcmin) & 30.8& 17.6&   2.9&    2.8&    9.8&    23.6""")

In [ ]:
for d in s4:
    if d["telescope"] == "SAT":
        d["pole_pol_sensitivity_DSR"] = table21.loc[int(d["center_frequency"].value)]["$Q/U$ rms ($\mu$K-arcmin)"] * u.uK*u.arcmin
    else:
        if d["band"] != "ULFL1": # no 20GHz in Chile
            d["chile_temp_sensitivity_DSR"] = table23.loc[int(d["center_frequency"].value)]["White noise level for $TT$ ($\mu$K-arcmin)"]* u.uK*u.arcmin
            d["chile_pol_sensitivity_DSR"] = table23.loc[int(d["center_frequency"].value)]["White noise level $E$/$B$ ($\mu$K-arcmin)"]* u.uK*u.arcmin
        d["pole_pol_sensitivity_DSR"] = table22.loc[int(d["center_frequency"].value)]["White noise level $E$/$B$ ($\mu$K-arcmin)"]* u.uK*u.arcmin

# Create the astropy Table

In [ ]:
import astropy.units as u

In [ ]:
from astropy.table import QTable, Table

In [ ]:
s4[0]

In [ ]:
t = QTable(rows=s4, masked=True, names=list(s4[1]))
t.add_index("band")

In [ ]:
t

In [ ]:
t.loc["MFLS1"]["NET"]

In [ ]:
t.loc["MFLS1"]["NET"]

In [ ]:
t.meta["comments"] = ipac_warning

In [ ]:
t.write("../cmbs4_instrument_model.tbl", format="ascii.ipac", overwrite=True)

In [ ]:
c = QTable.read("../cmbs4_instrument_model.tbl", format="ascii.ipac" )

In [ ]:
c.add_index("band")

In [ ]:
c.loc["LFS1"]

In [ ]:
cat ../cmbs4_instrument_model.tbl